# Carregar Dados, Armazenamento e Formatos de Arquivos
Bernardo Pandolfi Costa (19207646)

In [1]:
import numpy as np
import pandas as pd

## Lendo e Escrevendo Dados em Formato de Texto

In [2]:
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


A função ```read_csv``` carrega dados de um arquivo, link ou objeto.

In [3]:
df = pd.read_table('examples/ex1.csv', sep=',')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


A função ```read_table``` também carrega dados de um arquivo, link ou objeto e usa tab ('\t') como delimitador padrão:

In [4]:
df = pd.read_table('examples/ex1.csv', sep=',')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
# printing the columns types
print (df.dtypes)

print('---')

#printing the object type
print (type(df.a))

a           int64
b           int64
c           int64
d           int64
message    object
dtype: object
---
<class 'pandas.core.series.Series'>


In [6]:
#setting the columns type

df2 = pd.read_csv('examples/ex1.csv', dtype={'a': int, 'b': float, 'c': float, 'd': int, 'message':str})
df2.dtypes

a            int32
b          float64
c          float64
d            int32
message     object
dtype: object

In [7]:
for i, v in df2.iterrows():
    print(type(v.a), type(v.b), type(v.c), type(v.d), type(v.message))

<class 'int'> <class 'float'> <class 'float'> <class 'int'> <class 'str'>
<class 'int'> <class 'float'> <class 'float'> <class 'int'> <class 'str'>
<class 'int'> <class 'float'> <class 'float'> <class 'int'> <class 'str'>


Um arquivo nem sempre terá uma linha de header. Para ler tal arquivo, podemos deixar que pandas dê nomes padrões às colunas ou especificar os nomes na função:

In [8]:
pd.read_csv('examples/ex2.csv', header=None)


,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
pd.read_csv('examples/ex2.csv', names=['pri', 'seg', 'ter', 'qua', 'message'])

,pri,seg,ter,qua,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suponha que queiramos que a coluna 'message' seja o índice do DataFrame retornado:

In [10]:
names = ['a', 'b', 'c', 'd', 'message']

pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


Caso queiramos formar índices hierarquicos para múltiplas colunas, podemos passar uma lista de números de coluna ou nomes:

In [11]:
parsed = pd.read_csv('examples/csv_mindex.csv',index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

Em alguns casos, uma tabela pode não ter um delimitador dixo, usando um espaço em branco ou outro padrão para separar os campos. Considere um arquivo de texto do tipo:

In [12]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

Nesses casos, podemos escolher uma expressão para o ```read_table```. Podemos usar a expressão ```\s+```, nos dando:

In [13]:
result = pd.read_table('examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Como tinha uma coluna a mais que o número de linhas de dados, ```read_table``` infere que a primeira coluna deve ser o índice do DataFrame neste caso especial.

Podemos pular algumas linhas com a opção ```skiprows```:

In [14]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Lidar com valores vazios é uma parte importante do processo. Dados perdidos acontecem ou porque não estão presentes ou porque estão marcados por um valor sentinela. Por padrão, pandas usa um conjunto de sentinelas comuns, como NA e NULL:

In [15]:
result = pd.read_csv('examples/ex5.csv')
result


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [16]:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Diferentes sentinelas NA podem ser especificados para cada coluna em um dict:

In [18]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## Lendo Arquivos de Texto em Pedaços

Quando processando arquivos muito grandes ou encontrando o conjunto certo de argumentos para processar um arquivo grande, pode ser útil ler apenas um pedaço pequeno do arquivo ou iterar o arquivo em pedaços menores.

In [19]:
pd.options.display.max_rows = 10

In [20]:
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [21]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


Para ler um arquivo em partes, especificamos o chunksize como o número de linhas a ser lido.

In [22]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

O objeto TextParser retornado pelo read_csv permite que iteremos partes de um arquivo de acordo com o chunksize. Por exemplo, podemos iterar o ex7.csv agregando os valores da coluna 'key':

In [23]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot

C:\Users\butew\AppData\Local\Temp\ipykernel_19792\3005392887.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = pd.Series([])


0    151.0
1    146.0
2    152.0
3    162.0
4    171.0
     ...  
V    328.0
W    305.0
X    364.0
Y    314.0
Z    288.0
Length: 36, dtype: float64

In [24]:
tot = tot.sort_values(ascending=False)
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Escrevendo Dados em Formato de Texto

Dados podem ser exportados em formatos delimitados. Considere um dos csv lidos anteriormente:

In [25]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Usando o método ```to_csv``` no DataFrame, podemos escrever os dados em um arquivo separado por vírgulas.

In [26]:
data.to_csv('examples/out.csv')

Outros delimitadores podem ser usados. Escrever ```sys.stdout``` imprime o texto resultante no console:

In [27]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Valores perdidos aparecem como strings vazias no resultado. Podemos substituir isso com um outro valor sentinela:

In [28]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


Sem outras opções especificadas, ambas os labes das linhas e colunas são escritas. Isso pode ser desativado:

In [29]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


Podemos escrever apenas um subconjunto de colunas na ordem pedida:

In [30]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


Series também tem o método ```to_csv```:

In [31]:
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('examples/tseries.csv')

### Trabalhando com Formatos Delimitados

É possível carregar a maioria dos dados tabulares do disco usando funções como ```pandas.read_table```. Em alguns casos, porém, algum processamento manual pode ser necessário.

Não é incomum receber arquivos com um o mais linhas malformadas que atrapalham no ```read_table```.

Para ilustrar, considere um pequeno arquivo csv:

"a","b","c"

"1","2","3"

"1","2","3"

Para qualquer arquivo com um delimitador de um caractere, podemos usar o módulo de csv original do Python. Para isso, passamos um arquivo ou objeto aberto para o ```csv.reader```:

In [32]:
import csv

f = open('examples/ex7.csv')

reader = csv.reader(f)

In [33]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


Agora, é trabalho do programador transformar esta informação em um DataFrame, por exemplo:

Podemos ler um arquivo como uma lista de linhas:

In [34]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

Então, podemos separar as linhas na linha do header e nas linhas de dados:

In [35]:
header, values = lines[0], lines[1:]
values

[['1', '2', '3'], ['1', '2', '3']]

Agora podemos criar um dict de colunas de dados zip, que faz a transposição de linhas em colunas:

In [36]:
data_dict = {k: v for k, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

Por fim, podemos passar a um DataFrame:

In [37]:
pd.DataFrame(data_dict)

,a,b,c
0,1,2,3
1,1,2,3


In [38]:
# ZIP
#list of tuples:
seq1 = ['foo', 'bar', 'baz']
seq2 = ['one', 'two', 'three']
zipped = zip(seq1, seq2)
list(zipped)


[('foo', 'one'), ('bar', 'two'), ('baz', 'three')]

In [39]:
# zip can take an arbitrary number of sequences, and the number of elements it produces is determined by the shortest sequence:
seq3 = [False, True]
list(zip(seq1, seq2, seq3))


[('foo', 'one', False), ('bar', 'two', True)]

## Dados JSON

JSON se tornou um dos formatos mais famosos para entregar dados para requisitos HTTP entre browsers de web e aplicações. É uma forma muito mais livre que o csv.

Para converter JSON para a forma de Python (dict), usamos ```json.loads```:

In [40]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [41]:
import json

result = json.loads(obj)
print (type(result))
result

<class 'dict'>


{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

```json.dumps``` converte um objeto de Python de volta a JSON:

In [42]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

Coomo converter um objeto JSON ou lista de objetos para um DataFrame ou outra estrutura de dados para análise será por cargo do programador.

Convenientemente, podemos passar uma lista de dicts (que anteriormente eram objetos JSON) a um construtor de DataFrame e selecionar um subconjunto de campos de dados:

In [43]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


O ```pandas.read_json``` pode automaticamente converter JSON em arranjos específicos em Séries ou DataFrame. Por exemplo:

[{"a": 1, "b": 2, "c": 3},

{"a": 4, "b": 5, "c": 6},

{"a": 7, "b": 8, "c": 9}]

A opçõa padrão para ```pandas.read_json``` assume que cada objeto na array JSON é uma linha da tabela:

In [44]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


Caso seja preciso exportar dados de pandas para JSON, uma maneira é usar o método ```to_json``` em Séries e DataFrame:

In [45]:
s = data.to_json()
s

'{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}'

In [46]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


In [47]:
with open('data.txt', 'w') as outfile:  
    json.dump(s, outfile)

## HTML e XML

pandas Tem uma função pré-construída, ```read_html```, que usa bibliotecas como lxml e Beautiful Soup para autimaticamente parsear tabelas de arquivos HTML como objetos DataFrame.

In [48]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')
tables


[                             Bank Name             City  ST   CERT  \
 0                          Allied Bank         Mulberry  AR     91   
 1         The Woodbury Banking Company         Woodbury  GA  11297   
 2               First CornerStone Bank  King of Prussia  PA  35312   
 3                   Trust Company Bank          Memphis  TN   9956   
 4           North Milwaukee State Bank        Milwaukee  WI  20364   
 ..                                 ...              ...  ..    ...   
 542                 Superior Bank, FSB         Hinsdale  IL  32646   
 543                Malta National Bank            Malta  OH   6629   
 544    First Alliance Bank & Trust Co.       Manchester  NH  34264   
 545  National State Bank of Metropolis       Metropolis  IL   3815   
 546                   Bank of Honolulu         Honolulu  HI  21029   
 
                    Acquiring Institution        Closing Date  \
 0                           Today's Bank  September 23, 2016   
 1              

In [49]:
print (type(tables))
print (len(tables))

<class 'list'>
1


In [50]:
failures = tables[0] #first dataframe = tabela
print(type(failures))
failures.head()

<class 'pandas.core.frame.DataFrame'>


,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [51]:
print(failures.dtypes)

Bank Name                object
City                     object
ST                       object
CERT                      int64
Acquiring Institution    object
Closing Date             object
Updated Date             object
dtype: object


Podemos proceder a fazer uma análise e limpeza de dados, como computando o número de falhas bancárias por ano:

In [52]:
close_timestamps = pd.to_datetime(failures['Closing Date'])

print(type(close_timestamps))

close_timestamps  

<class 'pandas.core.series.Series'>


0     2016-09-23
1     2016-08-19
2     2016-05-06
3     2016-04-29
4     2016-03-11
         ...    
542   2001-07-27
543   2001-05-03
544   2001-02-02
545   2000-12-14
546   2000-10-13
Name: Closing Date, Length: 547, dtype: datetime64[ns]

In [53]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

In [54]:
# loading URL

tables2 = pd.read_html('https://pt.wikipedia.org/wiki/Lista_de_pa%C3%ADses_por_popula%C3%A7%C3%A3o')
# match='Municípios mais populosos'

print(len(tables2))

tables2[0]

2


,Posição,País (ou território dependente),Estimativa da ONU,Data,Estimativa Oficial
0,1,China,1 411 780 000,2021,Censo oficial
1,2,Índia,1 380 004 385,2020,Estimativa oficial
2,3,Estados Unidos,331 449 281,2020,Censo oficial
3,4,Indonésia,273 523 615,2020,Estimativa oficial
4,5,Paquistão,220 892 340,2020,Estimativa oficial
...,...,...,...,...,...
245,–,Ilha de Ascensão (Reino Unido),806,2021,Estimativa oficial[10]
246,195,Vaticano,801,2020,Estimativa oficial
247,–,Ilhas Cocos (Keeling) (Austrália),573,2020,Estimativa oficial
248,–,Tristão da Cunha (Reino Unido),241,2022,Estimativa oficial[11]


## Lendo Arquivos Excel

pandas também suporta ler dados tabulados armazenados em Excel 2003 ou mais atual usando a classe ExcelFile ou a função ```pandas.read_excel```.

In [55]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')
xlsx

In [56]:
#index_col: default None

pd.read_excel(xlsx, 'Sheet1', index_col=0)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [57]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


Para escrever dados de pandas ao formato Excel, criamos primeiro um ExcelWriter, então, escrevemos os dados usando o método ```to_excel```:

In [58]:
writer = pd.ExcelWriter('examples/ex2.xlsx')

frame.to_excel(writer, 'Sheet1')
writer.save()

Podemos também passar o caminho do arquivo ao ```to_excel``` e evitar o ExcelWriter:

In [59]:
frame.to_excel('examples/ex2.xlsx')

## Interagindo com APIs

Muitos sites têm APIs públicas que providenciam dados via JSON ou algum outro formato. Existem diversas maneiras para acessar essas APIs com Python. Um dos mais fáceis é o pacote <b>requests</b>.

In [60]:
import requests

url = 'https://api.github.com/repos/pandas-dev/pandas/issues?page=1&per_page=100'
resp = requests.get(url)
resp

<Response [200]>

In [61]:
data = resp.json()
data

[{'url': 'https://api.github.com/repos/pandas-dev/pandas/issues/48278',
  'repository_url': 'https://api.github.com/repos/pandas-dev/pandas',
  'labels_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/48278/labels{/name}',
  'comments_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/48278/comments',
  'events_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/48278/events',
  'html_url': 'https://github.com/pandas-dev/pandas/pull/48278',
  'id': 1352694726,
  'node_id': 'PR_kwDOAA0YD84948Ku',
  'number': 48278,
  'title': 'CI: Catch importwarning _SixMetaPathImporter.find_spec',
  'user': {'login': 'phofl',
   'id': 61934744,
   'node_id': 'MDQ6VXNlcjYxOTM0NzQ0',
   'avatar_url': 'https://avatars.githubusercontent.com/u/61934744?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/phofl',
   'html_url': 'https://github.com/phofl',
   'followers_url': 'https://api.github.com/users/phofl/followers',
   'following_url': 'https://api.github.

In [62]:
len(data)

100

In [63]:
data[0]['title']

'CI: Catch importwarning _SixMetaPathImporter.find_spec'

Cada elemento no dado é um dicionário contendo todos os dados encontrados em uma página de problemas do GitHub, com exceção dos comentários. Podemos passar esses dados diretamente a um DataFrame e extrair os campos de interesse:

In [64]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,48278,CI: Catch importwarning _SixMetaPathImporter.f...,[],open
1,48277,DEPR: Compatibility with NumPy in ops,"[{'id': 76865106, 'node_id': 'MDU6TGFiZWw3Njg2...",open
2,48276,Backport PR #48272 on branch 1.5.x (CI: Requir...,"[{'id': 48070600, 'node_id': 'MDU6TGFiZWw0ODA3...",open
3,48275,BUG: Fixed inconsistent multiplication behavio...,"[{'id': 47223669, 'node_id': 'MDU6TGFiZWw0NzIy...",open
4,48271,CI: newer version of s3fs is pulled causing bu...,"[{'id': 48070600, 'node_id': 'MDU6TGFiZWw0ODA3...",open
...,...,...,...,...
95,48014,Update pyarrow dependency,"[{'id': 527603109, 'node_id': 'MDU6TGFiZWw1Mjc...",open
96,48012,WEB: Standardize website/docs footer,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
97,48004,SAS7BDAT parser: Fix page count,"[{'id': 258745163, 'node_id': 'MDU6TGFiZWwyNTg...",open
98,48000,ENH: improve `Period` parsing,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open


## Interagindo com Banco de Dados

Carregar dados de SQL em um DataFrame é bem simples, e pandas possui algumas funções para simplificar ainda mais o processo.

Por exemplo, criaremos um banco de dados SQLite usando o driver pré-disponibilizado do Python sqlite3: 

In [65]:
import sqlite3

query = """
CREATE TABLE test4
(a VARCHAR(20),
 b VARCHAR(20),
 c REAL,
 d INTEGER
);"""

con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

In [66]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

stmt = "INSERT INTO test4 VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [67]:
cursor = con.execute('select * from test4')
print (cursor)
rows = cursor.fetchall()
print(type(rows))
rows

<class 'list'>


[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

A maioria dos drives Python SQL retornam uma lista de tuplas quando selecionando dados de uma tabela. Podemos passar a lista de tuplas a um construtor de DataFrame, mas também precisamos de nomes às colunas:

In [68]:
cursor.description


(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [69]:
# list comprehension x[0] for x in cursor.description

pd.DataFrame(rows, columns = [x[0] for x in cursor.description] )

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


O projeto SQLAlchemy é uma ferramenta popular para Python SQL que abstrai muitas das diferenças comuns entre bancos de dados SQL. pandas tem uma função ```read_sql``` que permite ler dados facilmente de uma conexão SQLAlchemy.

In [70]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

OperationalError: (sqlite3.OperationalError) no such table: test
[SQL: select * from test]
(Background on this error at: https://sqlalche.me/e/14/e3q8)